In [1]:
import h5py
import numpy as np
import pandas as pd

In [6]:
lfp_file = h5py.File('datasets/GRat31_636061_lfp_power.hdf5', 'r')
lfp_data = lfp_file['lfp_power']
print('lfp_file: ', lfp_data.shape)

lfp_file:  (384, 80690)


In [19]:
spikes_file = h5py.File('datasets/GRat31_636061_all_sorted_spikes.hdf5', 'r')
spikes_data = np.asarray(spikes_file['sorted_spikes']).T
all_neural_data = np.concatenate((lfp_data, spikes_data), axis=0)
print(all_neural_data.shape)

# lfp_data = lfp_file['lfp_power']
# print('lfp_file: ', lfp_data.shape)

(418, 80690)


In [ ]:
spike_fs = 3e4

def get_spike_times(ch):
    spiketimes_file = './SpikeTimes/ChGroup_%d/SpikeTimes' % ch

    spiketimes_file = open(spiketimes_file,'rb')
    # chgroup_0_spike_times = fread(spiketimes_file,inf,'uint64=>uint64');

    spike_times = np.fromfile(spiketimes_file,dtype=np.uint64) ## in samples
    spiketimes_file.close()

    last_spiketime = spike_times[-1]  ## in samples

    #print 'last_spiketime = ', last_spiketime
    win = 5. # sec
    step = 1. # sec 

    center = (win * spike_fs)/2 + 100
    win_times = []
    windowed_rates = []
    #print center / spike_fs,center+(win * spike_fs)/2 - 100
    while center+(win * spike_fs)/2 - 100  <= last_spiketime:
        win_times.append(center)
        #print center-(win * lfp_fs)/2 - 1, center, center+(win * lfp_fs)/2 - 1
        start = int(center-(win * spike_fs)/2 - 100) 
        stop = int(center+(win * spike_fs)/2 - 100) 
        
        num_spikes_in_window = len(spike_times[(spike_times < stop) & (spike_times > start)])
        
        rate = num_spikes_in_window / win 
        
        windowed_rates.append(rate)

        #print start, stop,num_spikes_in_window

        center += step*spike_fs
        
        
    return win_times,windowed_rates

win_times,windowed_rates = get_spike_times(0)